In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#my datset
m8 = pd.read_csv("train.csv")
import warnings
warnings.filterwarnings('ignore')

In [3]:
m8.head()

,Unnamed: 0,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,134857,43127,21.0,1,1,0,0,0,0,0.0,...,32.0,0.0,0.0,0,0,0.0,0.0,0,0,0
1,604,60250,61.0,1,1,0,0,0,0,0.0,...,32.0,0.0,0.0,0,0,0.0,0.0,0,0,0
2,792310,80,1.0,2,0,0,0,0,0,0.0,...,32.0,0.0,0.0,0,0,0.0,0.0,0,0,4
3,659776,54446,67.0,1,1,0,0,0,0,0.0,...,32.0,0.0,0.0,0,0,0.0,0.0,0,0,0
4,3160,53,32545.0,2,2,52,180,26,26,26.0,...,32.0,0.0,0.0,0,0,0.0,0.0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split
x = m8.iloc[:, :-1]
y = m8.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)

In [5]:
class_probabilities = {}
total_samples = len(y_train)

for class_name in np.unique(y_train):
    class_count = len(y_train[y_train == class_name])
    class_probabilities[class_name] = class_count / total_samples

In [6]:
feature_probabilities = {}

for class_name in np.unique(y_train):
    # Select rows belonging to the current class
    rows = X_train[y_train == class_name]
    # Calculate mean and standard deviation for each feature
    means = rows.mean(axis=0)
    stds = rows.std(axis=0)
    # Store the mean and standard deviation in a dictionary
    feature_probabilities[class_name] = (means, stds)

In [7]:
def predict(X, class_probabilities, feature_probabilities):
    # Calculate the posterior probability for each class
    posteriors = {}
    for class_name in class_probabilities:
        class_prior = class_probabilities[class_name]
        means, stds = feature_probabilities[class_name]
        likelihood = np.exp(-(X - means) ** 2 / (2 * stds ** 2)) / (np.sqrt(2 * np.pi) * stds)
        posterior = np.log(class_prior) + np.log(likelihood.prod(axis=0))
        posteriors[class_name] = posterior
    # Return the class with the highest posterior probability
    return max(posteriors, key=posteriors.get)

In [8]:
predictions = []
for i in range(len(X_test)):
    prediction = predict(X_test.iloc[i], class_probabilities, feature_probabilities)
    predictions.append(prediction)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.199440682955549


In [10]:
print("Precision:", precision_score(y_test, predictions, average='macro'))

Precision: 0.31571596774906296


In [11]:
print("Recall:", recall_score(y_test, predictions, average='macro'))

Recall: 0.4300495573779887


In [12]:
print("F1 score:", f1_score(y_test, predictions, average='macro'))

F1 score: 0.17973172077702393
